In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import src.Funciones as fun
from functools import reduce
import operator
from pandas import json_normalize
import numpy as np
from geopy.geocoders import Nominatim
import folium
from folium import Choropleth, Circle, Marker, Icon, Map

In [2]:
geolocator = Nominatim(user_agent="edu")
location = geolocator.geocode("barcelona")
lat = location.latitude
lon = location.longitude

In [3]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

In [4]:
queries_HS = ["Historic Site"]
queries_museum = ["Museum"]
queries_bar = ["Nightclub"]
queries_food = ["Food"]
queries_hotel = ["Hotel"]

In [5]:
tok1= os.getenv("Client_Id")
tok2= os.getenv("Client_Secret")

In [6]:
HS = fun.get_data(lat, lon,url_query,*queries_HS)
museum = fun.get_data(lat, lon,url_query,*queries_museum)
nightclub = fun.get_data(lat, lon,url_query,*queries_bar)
food = fun.get_data(lat, lon,url_query,*queries_food)
hoteles = fun.get_data(lat, lon,url_query,*queries_hotel)

In [7]:
HS = HS.get("Historic Site").get("response").get("groups")[0].get("items")
museum = museum.get("Museum").get("response").get("groups")[0].get("items")
nightclub = nightclub.get("Nightclub").get("response").get("groups")[0].get("items")
food = food.get("Food").get("response").get("groups")[0].get("items")
hoteles = hoteles.get("Hotel").get("response").get("groups")[0].get("items")

In [8]:
df_HS = fun.crear_dataframe(HS)
df_museum = fun.crear_dataframe(museum)
df_nightclub = fun.crear_dataframe(nightclub)
df_food = fun.crear_dataframe(food)
df_hoteles = fun.crear_dataframe(hoteles)

In [9]:
df_HS = df_HS[:10]
df_HS = (df_HS.assign(categoria = "historic site"))

In [10]:
df_hoteles = df_hoteles[:10]
df_hoteles = (df_hoteles.assign(categoria = "hotels"))

In [11]:
df_food = df_food[:10]
df_food = (df_food.assign(categoria = "food"))

In [12]:
df_museum = df_museum[:10]
df_museum = (df_museum.assign(categoria = "museum"))

In [13]:
df_nightclub = df_nightclub[:10]
df_nightclub = (df_nightclub.assign(categoria = "nightclub"))

In [21]:
df = pd.concat([df_food,df_hoteles,df_HS,df_museum,df_nightclub],ignore_index=True)

In [15]:
direcc = []
for i, row in df.iterrows():
    lat = row["latitud"]
    lon = row["longitud"]
    location = geolocator.reverse(f'{lat}'"," f'{lon}')
    calle = location.address
    direcc.append(calle)

In [30]:
df["Address"] = direcc
df_reducido = df[["name","Address"]]
df_reducido.head(10)

,name,Address
0,El Cuiner de Damasc,"2, Carrer dels Templers, el Gòtic, Ciutat Vell..."
1,Bodega La Palma,"10, Carrer de la Palma de Sant Just, el Gòtic,..."
2,Frankfurt Sant Jaume,"Frankfurt Sant Jaume, 2, Plaça de Sant Jaume, ..."
3,Brugarol Barcelona,"Green Bamboo, 10, Carrer de Salomó ben Adret, ..."
4,La Alcoba Azul,"5, Placeta de Manuel Ribé, el Gòtic, Ciutat Ve..."
5,pizzeria da nanni,"pizzeria da nanni, Carrer de la Llibreteria, e..."
6,L'antic Bocoi del Gòtic,"Green Bikes, 3, Plaça dels Traginers, el Gòtic..."
7,Pla Restaurant,"4, Carrer de Bellafila, el Gòtic, Ciutat Vella..."
8,Dibar Café,"3, Baixada de la Llibreteria, el Gòtic, Ciutat..."
9,Sensi Bistro,"4, Carrer del Regomir, el Gòtic, Ciutat Vella,..."


In [31]:
geolocator = Nominatim(user_agent="edu")
location = geolocator.geocode("barcelona")
lat = location.latitude
lon = location.longitude

In [32]:
map_2 = Map(location=[f'{lat}',f'{lon}'],zoom_start=14.5)

In [34]:
for i,row in df.iterrows():
    
    distrito = {
    "location" : [row["latitud"],row["longitud"]],
    "tooltip" : row["name"]
    }
    if row["categoria"] == "historic site":
        icon = Icon(color = "lightblue",
                prefix = "fa",
                icon = "eye",
                icon_color = "black")
        
                    
    elif row["categoria"] == "nightclub":
        icon = Icon(color = "lightred",
                prefix = "fa",
                icon = "glass",
                icon_color = "black")
                    
    elif row["categoria"] == "food":
        icon = Icon(color = "beige",
                prefix = "fa",
                icon = "cutlery",
                icon_color = "black")
        
    elif row["categoria"] == "hotels":
        icon = Icon(color = "cadetblue",
                prefix = "fa",
                icon = "h-square",
                icon_color = "white")
                    
    else: 
        icon = Icon(color = "lightblue",
                prefix = "fa",
                icon = "eye",
                icon_color = "black")
    Marker(**distrito, icon = icon).add_to(map_2)

In [35]:
map_2

## Mapa

In [23]:
map_2 = Map(location=[f'{lat}',f'{lon}'],zoom_start=14.5)

In [24]:
for i, row in df_nightclub.iterrows():
    distrito = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip" : row["name"]}
    icon = Icon(color = "lightred",
                prefix = "fa",
                icon = "glass",
                icon_color = "black")
    Marker(**distrito,icon = icon ).add_to(map_2)

In [25]:
for i, row in df_food.iterrows():
    distrito = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip" : row["name"]}
    icon = Icon(color = "beige",
                prefix = "fa",
                icon = "cutlery",
                icon_color = "black")
    Marker(**distrito,icon = icon ).add_to(map_2)

In [26]:
for i, row in df_hoteles.iterrows():
    distrito = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip" : row["name"]}
    icon = Icon(color = "cadetblue",
                prefix = "fa",
                icon = "h-square",
                icon_color = "white")
    Marker(**distrito,icon = icon ).add_to(map_2)

In [27]:
for i, row in df_HS.iterrows():
    distrito = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip" : row["name"]}
    icon = Icon(color = "lightblue",
                prefix = "fa",
                icon = "eye",
                icon_color = "black")
    Marker(**distrito,icon = icon ).add_to(map_2)

In [28]:
for i, row in df_museum.iterrows():
    distrito = {
        "location":[row["latitud"], row["longitud"]],
        "tooltip" : row["name"]}
    icon = Icon(color = "lightblue",
                prefix = "fa",
                icon = "eye",
                icon_color = "black")
    Marker(**distrito,icon = icon ).add_to(map_2)

In [29]:
map_2